In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import HashingVectorizer, ENGLISH_STOP_WORDS

from xgboost import XGBRegressor, DMatrix

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv('reviews_train.csv')
df_test = pd.read_csv('reviews_test.csv')
df_train.head(3)

,ID,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5.0,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4.0,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4.0,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600


Выделим текстовые поля и целевую переменную. Заполним пропуски.

In [3]:
X_train = df_train.summary.replace({np.nan: ''}) + ' ' + df_train.reviewText.replace({np.nan: ''})
X_test = df_test.summary.replace({np.nan: ''}) + ' ' + df_test.reviewText.replace({np.nan: ''})
y_train = df_train.overall

Сделаем базовую предобработку и снизим размерность при помощи hashing trick.

In [4]:
vectorizer = HashingVectorizer(stop_words=ENGLISH_STOP_WORDS, n_features=5000)

X_train_oh = vectorizer.fit_transform(X_train)
X_test_oh = vectorizer.transform(X_test)

Обучим xgBoost-регрессор.

In [5]:
regressor = XGBRegressor(max_depth=4, n_estimators=300, objective='reg:linear',
                         seed=42, nthread=4)
regressor.fit(X_train_oh, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=300, nthread=4,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=42, silent=True, subsample=1)

In [6]:
y_pred = regressor.predict(X_test_oh)

Сделаем сабмит.

In [7]:
pd.DataFrame(data={'ID': df_test.ID, 'overall': y_pred}).to_csv('submission.csv', index=False)